In [345]:
from importlib import reload
import honours_project.simulation as simulation
reload(simulation)
import honours_project.data as data
reload(data)

import json

In [350]:
from honours_project.data import PNE, Cycle, SimulationResult
from honours_project.simulation import Simulation

# Example of problematic seed
sim = simulation.Simulation(5, 5)
result1 = sim.run()
result2 = sim.run()
print(json.dumps(result1, indent=4, cls=data.EnhancedJSONEncoder))
print(json.dumps(result2, indent=4, cls=data.EnhancedJSONEncoder))

{
    "time": 0.551266416994622,
    "iteration": 283
}
{
    "time": 0.4019492499937769,
    "iteration": 223
}
